#  Final Project Submission

Please fill out:
* Student name: Erick Mauti, Samuel Ongechi, Isaac Onyango, Marilyn Akinyi, Rose Miriti , Rogers Otieno
* Student pace:  Part time 
* Instructor name: George Kamundia



# Strategic Movie Production: Market Analysis & Insights

##  Objective
To empower a new movie studio entering the film industry with no prior production experience with data-driven insights to guide its film selection and market entry strategy.  
This project aims to identify patterns in successful movies by analyzing existing industry data, focusing on key drivers such as **genre performance**, **production budgets**, **audience and critic reception**, and **release timing**.  
The ultimate goal is to deliver **three actionable business recommendations** to inform the studio’s decisions on **what types of films to produce** and **how to position them for commercial success**.

---

##  Business Problem
The film industry is inherently high-risk. Studios frequently invest millions without fully understanding the factors that drive profitability. For a new studio, this challenge is even greater due to the lack of historical performance data and industry experience.

Key questions often remain unanswered:
- Which genres have the best return on investment?
- What budget ranges balance cost and revenue most effectively?
- Do critic and audience ratings reliably indicate financial success?
- Is there an optimal release window for certain types of films?

This project addresses these challenges by consolidating and analyzing real-world data from multiple sources to uncover what makes a movie commercially viable. The resulting insights will help minimize financial risk and inform smarter strategic decisions in film production and distribution.

---

## Datasets Used

This project brings together multiple publicly available datasets, each offering a unique perspective on the movie industry:

- **The Numbers (`tn.movie_budgets.csv`)**  
  → Primary source for financial performance metrics

- **TMDb (`tmdb.movies.csv`)**   
  → Used for genre classification and audience sentiment

- **IMDb (SQLite DB)**  
  - `movie_basics`: Titles, years, runtime, genres  
  - `movie_ratings`: IMDb average ratings and vote counts  
  → Source of critic/audience evaluation and genre data

- **Rotten Tomatoes (`rt.movie_info.tsv`)**  
  → Used for supplementary analysis and genre validation

- **Box Office Mojo (`bom.movie_gross.csv`)**  
  → Serves as a backup for revenue and studio-level insights

Together, these datasets allow for a holistic exploration of the movie landscape—blending financials, content attributes, timing, and audience feedback into one cohesive analysis framework.


In [64]:
# Importing the necessary libraries needed to load, clean , analyze and visulize the data 

import pandas as pd
import sqlite3 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# 2. Data Loading and Initial Inspection

In this section, we load all the provided datasets. For each dataset, we will perform an initial inspection to understand its structure, dimensions, data types, and identify any immediate quality issues like a high number of missing values. This foundational understanding is critical before proceeding to data cleaning and merging.

In [65]:
#Connecting to the database
conn = sqlite3.connect('Data\im.db')

#Viewing the tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [66]:
""" ## Data Mining & Manipulation
# We systematically extracted data from im.db by joining these key tables:
1. movie_basics (core metadata: titles, years, runtimes, genres)
2. movie_ratings (IMDB ratings and vote counts)
3. movie_akas (unique movie_id for creating relationship). Serves as a critical bridge for creating accurate relationships between movies and their associated entities (directors, ratings, etc.).
4. directors (movie-director mappings)
5. persons (director names and details)

# Optimized SQL Approach:
1. Used CTEs (Common Table Expressions) to modularize the query:
First CTE cleaned director data by removing duplicates and joining with persons to get names.
Second CTE combined movie basics with ratings, filtering out NULL runtimes early to improve performance.
2. Employed JOIN to preserve movies with all the rquiered information for analysis.
3. Added explicit column aliases (e.g., primary_title AS title) for readability."""


# Loading and Joing tables of Interest of our analysis from the im.db database.
#__
combined_data = """
WITH movie_persons AS ( 
    -- the following code gets all the directors matched up with their movies
    SELECT 
        DISTINCT(a.movie_id) AS movie_id, -- handles duplicate records by returning only the unique ones
        b.person_id,
        c.primary_name AS person_name
    FROM 
        movie_akas a
    JOIN
        directors b
    USING(movie_id) -- links the movie_akas and directors using movie_id (common unique key)
    JOIN
        persons c -- from this table we get the director's details
    ON b.person_id = c.person_id -- Match the director ID to person record
),
combined_genre_rating AS (
    -- below code consolidates all the movie details and ratings as a table
    -- it also skips movies without runtime since they are incomplete records
    SELECT 
        a.genres,
        a.movie_id,
        a.start_year,
        a.primary_title AS title,
        a.runtime_minutes,
        b.averagerating,
        b.numvotes
    FROM 
        movie_basics a
    JOIN
        movie_ratings b
    USING(movie_id)
    WHERE 
        a.runtime_minutes IS NOT NULL -- excludes records without runtime
)
-- this combines directors' records with movie details and ratings 
SELECT
    a.genres,
    a.movie_id,
    a.start_year,
    a.title,
    a.runtime_minutes,
    a.averagerating,
    a.numvotes,
    b.person_id,
    b.person_name
FROM 
    combined_genre_rating a
JOIN
    movie_persons b
USING(movie_id)
WHERE a.genres IS NOT NULL -- filters out any movies without genre data
ORDER BY 
    a.start_year DESC -- sorts everything by release year (newest first)
"""
df1 = pd.read_sql(combined_data, conn) # reads the above sql code and tranforms it into a pandas dataframe.
df1



,genres,movie_id,start_year,title,runtime_minutes,averagerating,numvotes,person_id,person_name
0,"Biography,Drama",tt0066787,2019,One Day Before the Rainy Season,114.0,7.2,43,nm0002411,Mani Kaul
1,"Action,Adventure,Sci-Fi",tt0437086,2019,Alita: Battle Angel,122.0,7.5,88207,nm0001675,Robert Rodriguez
2,"Action,Adventure,Comedy",tt0448115,2019,Shazam!,132.0,7.4,109051,nm2497546,David F. Sandberg
3,"Animation,Family,Fantasy",tt0765465,2019,The Legend of Secret Pass,92.0,4.9,331,nm0872045,Steve Trenbirth
4,"Biography,Comedy,Drama",tt0800325,2019,The Dirt,107.0,7.0,26876,nm0871860,Jeff Tremaine
...,...,...,...,...,...,...,...,...,...
72702,"Drama,Romance",tt7210246,2010,Kasle Choryo Mero Man,138.0,6.8,10,nm9174885,Shabir Shrestha
72703,Documentary,tt7502284,2010,David Bowie: Rare and Unseen,64.0,6.0,5,nm7128404,Paul Clark
72704,Horror,tt8090084,2010,Goodbye Dolly,50.0,6.8,5,nm1751788,Andrew N. Shearer
72705,Comedy,tt8147080,2010,Accusation,90.0,5.0,15,nm0267420,Mehrdad Farid


In [67]:
# Checking for nulls in df1
(df1.isnull().sum()/len(df1))*100

genres             0.0
movie_id           0.0
start_year         0.0
title              0.0
runtime_minutes    0.0
averagerating      0.0
numvotes           0.0
person_id          0.0
person_name        0.0
dtype: float64

In [68]:
# Closing Connection 

#conn.close()

In [69]:
# loading the bom.movie_gross csv

bom_movie_df = pd.read_csv(r"Data\bom.movie_gross.csv")
bom_movie_df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [70]:
# loading the rt_critic_review csv


rt_review_df = pd.read_csv(r"Data\rt_critic_reviews.csv")
rt_review_df

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...
...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme..."
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th..."


In [71]:
# loading the rt_movies csv

rt_movies_df = pd.read_csv(r"Data\rt_movies.csv")
rt_movies_df

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17707,m/zoot_suit,Zoot Suit,Mexican-American gangster Henry Reyna (Daniel ...,NaN,R,"Drama, Musical & Performing Arts",Luis Valdez,Luis Valdez,"Daniel Valdez, Edward James Olmos, Charles Aid...",1981-10-02,...,MCA Universal Home Video,Rotten,56.0,9.0,Upright,74.0,1195.0,2,5,4
17708,m/zootopia,Zootopia,From the largest elephant to the smallest shre...,The brilliantly well-rounded Zootopia offers a...,PG,"Action & Adventure, Animation, Comedy","Byron Howard, Rich Moore, Jared Bush","Jared Bush, Phil Johnston","J.K. Simmons, Kristen Bell, Octavia Spencer, A...",2016-03-04,...,Walt Disney Animation Studios,Certified-Fresh,98.0,291.0,Upright,92.0,101511.0,50,285,7
17709,m/zorba_the_greek,Zorba the Greek,Traveling to inspect an abandoned mine his fat...,NaN,NR,"Action & Adventure, Art House & International,...",NaN,NaN,"Anthony Quinn, Alan Bates, Irene Papas, Lila K...",1964-12-17,...,Fox,Fresh,80.0,10.0,Upright,86.0,7146.0,0,8,2
17710,m/zulu,Zulu,"In 1879, the Zulu nation hands colonial Britis...",Zulu patiently establishes a cast of colorful ...,PG,"Classics, Drama","Cy Endfield, Cyril Endfield","Cy Endfield, John Prebble","Stanley Baker, Jack Hawkins, Ulla Jacobsson, J...",1964-06-17,...,Paramount Pictures,Fresh,96.0,23.0,Upright,91.0,30193.0,6,22,1


In [72]:
# loading the tmdb.movies csv



tmdb_movies_df = pd.read_csv(r"Data\tmdb.movies.csv")
tmdb_movies_df

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [73]:
# loading the tn.movie_budget csv 

tn_budget_df = pd.read_csv(r"Data\tn.movie_budgets.csv")
tn_budget_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


## Data Preparation

After successfully loading the IMDb and CSV datasets, we start by exploring the structure and content of each dataset. This step helps us understand the type of information available, the quality of the data, and areas that may need cleaning before any meaningful analysis.

### Dataset Overview

We first check the shape of each dataset, the column names, data types, missing values, duplicate rows, and summary statistics using `.shape`, `.columns`, `.dtypes`, and `.describe()` methods.
To streamline this process, we created a function that inspects all datasets at once and returns a comprehensive overview for each, avoiding the need to check them one by one.


### Explanation

- The function `inspect_df` prints important information about each DataFrame.
- This includes shape, column names, data types, summary statistics, missing values, and duplicate counts.
- Running this function on each dataset helps us quickly identify data issues and decide cleaning steps.



In [74]:

#creating a function to inspect the data for our dataframe 
def inspect_df(df, name=None, preview_rows=5):

    # Show the name of the dataframe we are inspecting
    print(f"\n=== Inspecting: {name or 'DataFrame'} ===")
    
    # Print the shape of the DataFrame (rows, columns)
    print(f"Shape: {df.shape}")
    
    # Print list of column names
    print(f"Columns: {df.columns.tolist()}")
    
    # Print data types for each column
    print("\nData Types:")
    print(df.dtypes)
    
    # Print descriptive statistics for all columns
    print("\nDescriptive Statistics:")
    display(df.describe(include='all'))
    
    # Print the number of null values in each column
    print("\nMissing Values per Column:")
    print(df.isnull().sum())
    
    # Print the number of duplicate rows in the DataFrame
    print(f"\nDuplicate Rows: {df.duplicated().sum()}")

    
# Calling the Function with the different dataframes
inspect_df(bom_movie_df , name="bom_movie_df ")

inspect_df(rt_review_df , name="rt_review_df ")

inspect_df(rt_movies_df  , name="rt_movies_df")

inspect_df(tmdb_movies_df , name="tmdb_movies_df")

inspect_df(tn_budget_df, name="tn_budget_df")


=== Inspecting: bom_movie_df  ===
Shape: (3387, 5)
Columns: ['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']

Data Types:
title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object

Descriptive Statistics:


,title,studio,domestic_gross,foreign_gross,year
count,3387,3382,3.359000e+03,2037,3387.000000
unique,3386,257,NaN,1204,NaN
top,Bluebeard,IFC,NaN,1200000,NaN
freq,2,166,NaN,23,NaN
mean,NaN,NaN,2.874585e+07,NaN,2013.958075
std,NaN,NaN,6.698250e+07,NaN,2.478141
min,NaN,NaN,1.000000e+02,NaN,2010.000000
25%,NaN,NaN,1.200000e+05,NaN,2012.000000
50%,NaN,NaN,1.400000e+06,NaN,2014.000000
75%,NaN,NaN,2.790000e+07,NaN,2016.000000



Missing Values per Column:
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

Duplicate Rows: 0

=== Inspecting: rt_review_df  ===
Shape: (1130017, 8)
Columns: ['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date', 'review_content']

Data Types:
rotten_tomatoes_link    object
critic_name             object
top_critic                bool
publisher_name          object
review_type             object
review_score            object
review_date             object
review_content          object
dtype: object

Descriptive Statistics:


,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
count,1130017,1111488,1130017,1130017,1130017,824081,1130017,1064211
unique,17712,11108,2,2230,2,814,8015,949181
top,m/star_wars_the_rise_of_skywalker,Emanuel Levy,False,New York Times,Fresh,3/5,2000-01-01,Parental Content Review
freq,992,8173,841481,13293,720210,90273,48019,267



Missing Values per Column:
rotten_tomatoes_link         0
critic_name              18529
top_critic                   0
publisher_name               0
review_type                  0
review_score            305936
review_date                  0
review_content           65806
dtype: int64

Duplicate Rows: 119471

=== Inspecting: rt_movies_df ===
Shape: (17712, 22)
Columns: ['rotten_tomatoes_link', 'movie_title', 'movie_info', 'critics_consensus', 'content_rating', 'genres', 'directors', 'authors', 'actors', 'original_release_date', 'streaming_release_date', 'runtime', 'production_company', 'tomatometer_status', 'tomatometer_rating', 'tomatometer_count', 'audience_status', 'audience_rating', 'audience_count', 'tomatometer_top_critics_count', 'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count']

Data Types:
rotten_tomatoes_link                 object
movie_title                          object
movie_info                           object
critics_consensus                 

...


Missing Values per Column:
rotten_tomatoes_link                   0
movie_title                            0
movie_info                           321
critics_consensus                   8578
content_rating                         0
genres                                19
directors                            194
authors                             1542
actors                               352
original_release_date               1166
streaming_release_date               384
runtime                              314
production_company                   499
tomatometer_status                    44
tomatometer_rating                    44
tomatometer_count                     44
audience_status                      448
audience_rating                      296
audience_count                       297
tomatometer_top_critics_count          0
tomatometer_fresh_critics_count        0
tomatometer_rotten_critics_count       0
dtype: int64

Duplicate Rows: 0

=== Inspecting: tmdb_movies_df ===
Sh

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
count,26517.00000,26517,26517.000000,26517,26517,26517.000000,26517,26517,26517.000000,26517.000000
unique,NaN,2477,NaN,76,24835,NaN,3433,24688,NaN,NaN
top,NaN,[99],NaN,en,Eden,NaN,2010-01-01,Eden,NaN,NaN
freq,NaN,3700,NaN,23291,7,NaN,269,7,NaN,NaN
mean,13258.00000,NaN,295050.153260,NaN,NaN,3.130912,NaN,NaN,5.991281,194.224837
std,7654.94288,NaN,153661.615648,NaN,NaN,4.355229,NaN,NaN,1.852946,960.961095
min,0.00000,NaN,27.000000,NaN,NaN,0.600000,NaN,NaN,0.000000,1.000000
25%,6629.00000,NaN,157851.000000,NaN,NaN,0.600000,NaN,NaN,5.000000,2.000000
50%,13258.00000,NaN,309581.000000,NaN,NaN,1.374000,NaN,NaN,6.000000,5.000000
75%,19887.00000,NaN,419542.000000,NaN,NaN,3.694000,NaN,NaN,7.000000,28.000000



Missing Values per Column:
Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

Duplicate Rows: 0

=== Inspecting: tn_budget_df ===
Shape: (5782, 6)
Columns: ['id', 'release_date', 'movie', 'production_budget', 'domestic_gross', 'worldwide_gross']

Data Types:
id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object

Descriptive Statistics:


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
count,5782.000000,5782,5782,5782,5782,5782
unique,NaN,2418,5698,509,5164,5356
top,NaN,"Dec 31, 2014",Halloween,"$20,000,000",$0,$0
freq,NaN,24,3,231,548,367
mean,50.372363,NaN,NaN,NaN,NaN,NaN
std,28.821076,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN
25%,25.000000,NaN,NaN,NaN,NaN,NaN
50%,50.000000,NaN,NaN,NaN,NaN,NaN
75%,75.000000,NaN,NaN,NaN,NaN,NaN



Missing Values per Column:
id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

Duplicate Rows: 0


### Missing Data Threshold Check

Before cleaning,we decided to identify which columns have a high proportion of missing values. Columns with too many missing entries may need to be dropped or filled, as they could affect the quality of our analysis.

### Approach

We define a function `check_missing_threshold` that calculates the fraction of missing data per column in a DataFrame. It then returns a list of columns where the missing data exceeds a threshold of 30% (0.3).

The function is applied to each of our datasets to quickly identify problematic columns that may need to be dropped or filled.

In [75]:
#Defining the Function
def check_missing_threshold(df, threshold=0.3):

    # Calculate fraction of missing values per column
    missing_ratio = df.isnull().mean()
    
    # Select columns where missing ratio is above threshold
    cols_above_threshold = missing_ratio[missing_ratio > threshold].index.tolist()
    
    return cols_above_threshold


# load dataframes
dataframes = {
    "bom_movie_df ": bom_movie_df ,
    "rt_review_df": rt_review_df,
    "rt_movies_df": rt_movies_df,
    "tmdb_movies_df ": tmdb_movies_df ,
    "tn_budget_df": tn_budget_df,
}

# Check missing data in each DataFrame and print results
for name, df in dataframes.items():
    cols_missing = check_missing_threshold(df, threshold=0.3)
    if cols_missing:
        print(f"In '{name}' DataFrame, these columns have more than 30% missing data: {cols_missing}")
    else:
        print(f"In '{name}' DataFrame, no columns exceed 30% missing data.")

In 'bom_movie_df ' DataFrame, these columns have more than 30% missing data: ['foreign_gross']
In 'rt_review_df' DataFrame, no columns exceed 30% missing data.
In 'rt_movies_df' DataFrame, these columns have more than 30% missing data: ['critics_consensus']
In 'tmdb_movies_df ' DataFrame, no columns exceed 30% missing data.
In 'tn_budget_df' DataFrame, no columns exceed 30% missing data.


## Column Selection for Focused Analysis

To streamline our analysis, we narrowed down the datasets to include only the most relevant columns that capture essential aspects of movie performance, audience engagement, and financial outcomes.

For the **rt_movies_df** dataset, we selected the following columns based on relevance to our analysis genres, content_rating, tomatometer_status, tomatometer_rating, audience_rating, audience_count, tomatometer_count, movie_title, original_release_date and runtime . The **tmdb_movies_df** was reduced to key metrics on popularity and the original_language of the movies. Lastly, the **tn_budget_df** includes financial information such as the movie, production_budget, and both domestic_gross and worldwide_gross earnings.

This careful column selection allows us to focus our cleaning and analysis on the data that matters most for understanding what drives movie success.

In [76]:
# Defining columns to keep from the chosen CSV

selected_columns = {
    "rt_movies_df": ['genres', 'content_rating', 'tomatometer_status',
                     'tomatometer_rating', 'audience_rating', 'audience_count',
                     'tomatometer_count','movie_title', 'original_release_date','runtime'],
    
    "tmdb_movies_df": ['popularity', 'original_language'],
    
    "tn_budget_df": ['movie', 'production_budget', 'domestic_gross', 'worldwide_gross']
}

# Trim DataFrames to selected columns
rt_movies_df = rt_movies_df[selected_columns['rt_movies_df']]
tmdb_movies_df = tmdb_movies_df[selected_columns['tmdb_movies_df']]
tn_budget_df = tn_budget_df[selected_columns['tn_budget_df']]

#Calling the DataFrames
rt_movies_df
tmdb_movies_df
tn_budget_df


,movie,production_budget,domestic_gross,worldwide_gross
0,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...
5777,Red 11,"$7,000",$0,$0
5778,Following,"$6,000","$48,482","$240,495"
5779,Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,A Plague So Pleasant,"$1,400",$0,$0


## Data Cleaning

Cleaning each dataset individually seemed a bit tiresome and time consuuming, especially when working with multiple sources. To streamline the process, we created a reusable function that performs essential cleaning steps across all our selected DataFrames.

The cleaning function does the following:
- Strips whitespace from column names and string values,
- Converts any column containing the word “date” into a proper datetime format.

By applying this function to our chosen datasets **rt_movies_df**, **tmdb_movies_df**, and **tn_budget_df** we ensure consistency and save time. This also makes the data easier to work with in later stages of the project, such as merging and analysis.


In [77]:
#defining the cleaning Function 
def clean_dataframe(df):
    # Remove trailing spaces from column names
    df.columns = df.columns.str.strip()

    # Loop through each column in the DataFrame
    for col in df.columns:
        # If the column contains text, strip trailing spaces
        if df[col].dtype == 'object':
            df.loc[:, col] = df[col].str.strip()  # Use .loc to avoid SettingWithCopyWarning


    # Return the cleaned DataFrame
    return df

#Calling the Function
clean_dataframe(rt_movies_df)
clean_dataframe(tmdb_movies_df)
clean_dataframe(tn_budget_df)


,movie,production_budget,domestic_gross,worldwide_gross
0,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...
5777,Red 11,"$7,000",$0,$0
5778,Following,"$6,000","$48,482","$240,495"
5779,Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,A Plague So Pleasant,"$1,400",$0,$0


#### Column Name Standardization

To maintain consistency across all datasets, we standardized the column names by:
- Converting all letters to lowercase
- Stripping leading and trailing whitespace
- Replacing spaces with underscores

This ensures easier merging and manipulation of data in later stages.


In [78]:
# Define a function to standardize DataFrame column names
def standardize_dataframe(df):

    # Convert all column names to lowercase, strip whitespace, and replace spaces with underscores
    df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")
    return df

#Calling the Function
standardize_dataframe(tn_budget_df)
standardize_dataframe(tmdb_movies_df)
standardize_dataframe(rt_movies_df)

,genres,content_rating,tomatometer_status,tomatometer_rating,audience_rating,audience_count,tomatometer_count,movie_title,original_release_date,runtime
0,"Action & Adventure, Comedy, Drama, Science Fic...",PG,Rotten,49.0,53.0,254421.0,149.0,Percy Jackson & the Olympians: The Lightning T...,2010-02-12,119.0
1,Comedy,R,Certified-Fresh,87.0,64.0,11574.0,142.0,Please Give,2010-04-30,90.0
2,"Comedy, Romance",R,Fresh,67.0,53.0,14684.0,24.0,10,1979-10-05,122.0
3,"Classics, Drama",NR,Certified-Fresh,100.0,97.0,105386.0,54.0,12 Angry Men (Twelve Angry Men),1957-04-13,95.0
4,"Action & Adventure, Drama, Kids & Family",G,Fresh,89.0,74.0,68918.0,27.0,"20,000 Leagues Under The Sea",1954-01-01,127.0
...,...,...,...,...,...,...,...,...,...,...
17707,"Drama, Musical & Performing Arts",R,Rotten,56.0,74.0,1195.0,9.0,Zoot Suit,1981-10-02,104.0
17708,"Action & Adventure, Animation, Comedy",PG,Certified-Fresh,98.0,92.0,101511.0,291.0,Zootopia,2016-03-04,108.0
17709,"Action & Adventure, Art House & International,...",NR,Fresh,80.0,86.0,7146.0,10.0,Zorba the Greek,1964-12-17,142.0
17710,"Classics, Drama",PG,Fresh,96.0,91.0,30193.0,23.0,Zulu,1964-06-17,135.0


### Handling Missing Values

After inspecting each dataset using a missing data threshold of 30%, we identified that:

- The **foreign_gross** column in `bom_movie_df` and **critics_consensus** in `rt_movies_df` have more than 30% missing values.

For the selected columns, including **foreign_gross**, missing data still existed but was below the threshold. Instead of dropping rows or columns with missing values, we decided to fill missing values because:

- Dropping rows would reduce the dataset size and potentially remove valuable information, especially if missing data is scattered across many rows.

- Dropping columns would mean losing important features that contribute to understanding movie performance.

- Filling missing values allows us to keep the full dataset intact, preserving as much information as possible.

Specifically, we implemented a targeted filling strategy as follows:

- Missing categorical/text data was filled with `"Unknown"` to retain meaningful category information.

- Missing financial numeric columns (such as `production_budget`, `worldwide_gross`, `profit`, and `roi`) were filled with the median of each respective column.  We used the median because the data is rightly skewed and the mean would not serve the purpose as it was increased by a few outliers in the data.This approach helps avoid bias that could be introduced by filling with zero and better reflects typical financial values.

- Missing other numeric columns (non-financial) were filled with `0` to maintain numerical consistency and avoid errors during calculations.

This balanced strategy ensures that financial metrics maintain realistic central values while other numeric data remains consistent, preparing a clean and usable dataset for further analysis.

In [79]:
def fill_missing_values(df, financial_cols=None):
    # Initialize empty list for financial columns 
    financial_cols = financial_cols or []

    # Go through each column in the df
    for col in df.columns:
        if df[col].dtype == 'object':
            # If the column is categorical fill with "Unknown"
            df.loc[:, col] = df[col].fillna("Unknown")
        else:
            if col in financial_cols:
                # If it's a financial number, fill missing values with the median
                median_val = df[col].median()
                df.loc[:, col] = df[col].fillna(median_val)
            else:
                # For other number columns, fill missing values with 0
                df.loc[:, col] = df[col].fillna(0)

    return df

# List of financial columns in our df
financial_columns = ['production_budget', 'worldwide_gross', 'profit', 'ROI']

# Clean up missing data in all three datasets
fill_missing_values(rt_movies_df, financial_columns)
fill_missing_values(tmdb_movies_df, financial_columns)
fill_missing_values(tn_budget_df, financial_columns)

,movie,production_budget,domestic_gross,worldwide_gross
0,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...
5777,Red 11,"$7,000",$0,$0
5778,Following,"$6,000","$48,482","$240,495"
5779,Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,A Plague So Pleasant,"$1,400",$0,$0
